In [9]:
from pydub import AudioSegment
from pydub.playback import play

In [3]:
pl = AudioSegment.from_mp3('audio/forrest_audio_all.mp3')

# Удаление сцен

In [19]:
def timestamp_str_to_miliseconds(timestamp):
    
    def time_to_sec(time):
        time = time.split(':')
        return float(time[0]) * 60 * 60 + float(time[1]) * 60 + float(time[2])
    
    time_start, time_end = timestamp.split(' --> ')
    return int(time_to_sec(time_start) * 1000), int(time_to_sec(time_end) * 1000 + 1)

In [20]:
times_to_stay = [
'00:00:00.00 --> 00:21:32.12',
'00:24:13.24 --> 00:38:31.23', 
'00:38:58.20 --> 00:57:19.22', 
'00:59:31.17 --> 01:18:14.00', 
'01:20:24.16 --> 01:34:18.06', 
'01:37:14.19 --> 01:41:30.19',
'01:42:49.19 --> 02:09:51.17']

times_to_stay = map(timestamp_str_to_miliseconds, times_to_stay)

In [21]:
times_to_stay

[(0, 1292121),
 (1453240, 2311231),
 (2338200, 3439221),
 (3571170, 4694001),
 (4824160, 5658061),
 (5834190, 6090191),
 (6169190, 7791171)]

In [24]:
cut_audio = pl[times_to_stay[0][0]: times_to_stay[0][1]]

for time_frame in times_to_stay[1:]:
    cut_audio += pl[time_frame[0]:time_frame[1]]

In [34]:
print len(cut_audio) * 1. / 1000. / 60., ' min ', len(cut_audio), " miliseconds "

118.09745  min  7085847  miliseconds 


# Режем на сегменты

 Except for the first movie segment, each segment started with a movie snippet of at least six seconds immediately preceding the movie scene boundary used to split the segments. The start of each segment was synchronized with the MRI scanner’s volume acquisition trigger signal. All segments, except for the first one, started by repeating the equivalent of three fMRI volumes from the previous segment.

In [39]:
import numpy as np

segments_duration = [902., 882., 876., 976., 924., 878., 1084., 675.04] #in sec
cumsum_segments_time = np.cumsum(np.array(segments_duration)) * 1000 #in milisec
cumsum_segments_time

array([  902000.,  1784000.,  2660000.,  3636000.,  4560000.,  5438000.,
        6522000.,  7197040.])

In [40]:
pl[:cumsum_segments_time[0]].export('audio/forrest_segments/segment_1.mp3', 'mp3')

<open file 'audio/forrest_segments/segment_1.mp3', mode 'wb+' at 0x7fa318151660>

In [41]:
delay = 6000

In [43]:
for i in range(1, len(cumsum_segments_time)):
    pl[cumsum_segments_time[i-1] - delay: \
       cumsum_segments_time[i]].export('audio/forrest_segments/segment_{}.mp3'.format(i+1),
                                                                          'mp3')